In [1]:
%matplotlib widget
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import gsw

In [3]:
ctd_dir = 'data/CTD_data_Station_M/'

file_list = glob(ctd_dir+'*CtdDer.txt')

#filename = '2006347tibrCtdDer.txt'

In [4]:
mlat = 35+8.4585/60  
mlon = -122-59.9036/60

In [5]:
dz = 1
salspike_thresh = 0.2

In [30]:
plt.figure()
plt.gca().invert_yaxis()

dflist = []

for i,filename in enumerate(file_list):
    try:
        df = pd.read_csv(filename)
        
        
        #spi, = np.where((np.abs(np.diff(df['salin']))>0.2) | (np.abs(np.diff(df['pres']))>10))
        spi, = np.where((np.abs(np.diff(df['salin'][:-1]))>salspike_thresh) & 
                        (np.abs(np.diff(df['salin'][1:]))>salspike_thresh))
        spi = spi+1
        
        df['salin'][spi] = np.nan
        df['sigmat'][spi] = np.nan
        df['sigmatheta'][spi] = np.nan
        
        df['SA'] = gsw.SA_from_SP(df['salin'], df['pres'], mlon, mlat)
        df['CT'] = gsw.CT_from_t(df['SA'],df['temp'],df['pres'])
        df['sigma0'] = gsw.sigma0(df['SA'],df['CT'])
        df['sigma4'] = gsw.sigma4(df['SA'],df['CT'])
        
        df['zp'] = np.floor(df['pres']/dz+0.5)*dz
        dfm = df.groupby('zp').mean()
        
        dfm['SA'] = dfm['SA'].rolling(50,win_type='hanning').mean()
        dfm['CT'] = dfm['CT'].rolling(50,win_type='hanning').mean()
        #dfm['SA'] = dfm['sigmatheta'].rolling(25,win_type='hanning').mean()
        
        Nsq,pr_mid = gsw.Nsquared(dfm['SA'],dfm['CT'],dfm.index,lat=mlat)
        N = np.sqrt(Nsq)
        
        if i==0:
            dfall = dfm
        else:
            dfall = dfall.append(dfm)
        
        plt.semilogx(N,pr_mid)
    except:
        pass

FigureCanvasNbAgg()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tomconnolly/programs/anaconda/envs/py37/lib/pyth

In [14]:
dfmean = dfm.groupby('zp').mean()

plt.figure()
plt.plot(dfall['sigmatheta'],dfall.index,linewidth=1,color='gray')
plt.plot(dfmean['sigmatheta'],dfmean.index,linewidth=2,color='k')
plt.gca().invert_yaxis()

NameError: name 'dfm' is not defined

In [ ]:
f = gsw.f(35+8.4585/60)

sigmatheta = np.array(dfmean['sigmatheta'])
zp = np.array(dfmean.index)
drhodz = -(sigmatheta[2:]-sigmatheta[:-2])/(zp[2:]-zp[:-2])
z = 0.5*(zp[2:]+zp[:-2])

N = np.sqrt(-9.8*drhodz/1025)

In [ ]:
plt.figure()
plt.semilogx(-drhodz[:-1],z[:-1])
yl = plt.ylim()
plt.semilogx([f,f],yl,'k--')
plt.ylim(yl)
plt.gca().invert_yaxis()

In [ ]:
plt.figure()
plt.plot(-drhodz[:-1],z[:-1])
yl = plt.ylim()
plt.plot([f,f],yl,'k--')
plt.ylim(yl)
plt.gca().invert_yaxis()

In [ ]:
len(np.array(dfmean.index))

In [ ]:
plt.figure()
plt.hist(np.diff(df['salin']),100)

In [ ]:
dfm